In [ ]:
from google.colab import auth

auth.authenticate_user()

In [ ]:
!pip install --upgrade google-cloud-aiplatform==1.27.0 langchain==0.0.196 faiss-cpu==1.7.4 --user
!pip install transformers
!pip install -U sentence-transformers

In [ ]:
import os
import warnings
import torch
from transformers import LongT5Model, AutoTokenizer
from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformer, util
from langchain.llms import VertexAI
from copy import deepcopy


import vertexai
from sentence_transformers import SentenceTransformer, util

PROJECT_ID = "[delhivery-gen-ai-6]"  # @param {type:"string"}
vertexai.init(project='delhivery-gen-ai-6', location="us-central1")



from vertexai.preview.language_models import (TextEmbeddingModel,
                                              TextGenerationModel)

generation_model = TextGenerationModel.from_pretrained("text-bison@001")

warnings.filterwarnings("ignore")


In [ ]:
data_root = '/content/drive/MyDrive/Hackathon/train_dataset'

L1 = os.path.join(data_root, 'L1')


l1_files = os.listdir(L1)
l1_files_final = []

for file in l1_files:
  if file == '.ipynb_checkpoints':
    continue

  if '.' in file:
    file_name, _ = file.split('.')
    l1_files_final.append(file_name)
  else:
    l1_files_final.append(file)

l1_dict = dict({})
for file in l1_files_final:
  l1_dict[file] = open(os.path.join(L1, file+'.txt')).read()


L2 = os.path.join(data_root, 'L2')

l2_files = os.listdir(L2)
l2_dict = dict({})

for dir in l2_files:
  if dir == '.ipynb_checkpoints':
    continue
  files_in_dir = os.listdir(os.path.join(L2, dir))
  files_dict = dict({})
  for file in files_in_dir:
    if file == '.ipynb_checkpoints':
      continue
    file_content = open(os.path.join(L2, dir, file), 'r').read()
    files_dict[file.split('.')[0]] = file_content
  l2_dict[dir] = files_dict

data = {
    'L1' : l1_dict,
    'L2' : l2_dict
}

In [ ]:
# model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')
model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
def model_predict(str):
  embeddings = model.get_embeddings([str])
  return [each.values for each in embeddings][0]

In [ ]:
embedding_dict = deepcopy(data)

In [ ]:
for k, v in embedding_dict.items():
  if k == 'L1':
    for k1, v1 in v.items():
      print('processing ', k1)
      embedding_dict[k][k1] = model_predict(v1)

  else:
    for k1, v1 in v.items():
      for k2, v2 in v1.items():
        print('processing ', k2)
        embedding_dict[k][k1][k2] = model_predict(v2)

processing  overview
processing  exceptions_ndr
processing  support
processing  dispute
processing  information_center
processing  settings
processing  services
processing  orders
processing  onboarding
processing  overview
processing  domestic_support
processing  international_support
processing  domestic_order
processing  pickup_request
processing  international_order
processing  ndr
processing  exceptions
processing  channel_integration
processing  value_added_service
processing  logistic_services
processing  users
processing  configuration
processing  profile
processing  api_setup
processing  weight_mismatch
processing  resolving_claims
processing  calculating_volumetric_weight
processing  rate_calculator
processing  rate_card
processing  restricted_items
processing  reports


In [ ]:
import pickle

save_training_data_path = '/content/drive/MyDrive/Hackathon/save_vector'
save_corpus_path = os.path.join(save_training_data_path,'corpus.obj')
save_vector_path = os.path.join(save_training_data_path,'vector.obj')

if os.path.exists(save_corpus_path):
  os.remove(save_corpus_path)

fileObj = open(save_corpus_path, 'wb')
pickle.dump(data,fileObj)
fileObj.close()

if os.path.exists(save_vector_path):
  os.remove(save_vector_path)

fileObj = open(save_vector_path, 'wb')
pickle.dump(embedding_dict,fileObj)
fileObj.close()



Retrievel logic: takes query, embedding dataset, original dataset

In [ ]:
# threshold = 2, for max
def query_the_db(query, embedding_dict, data, threshold=0.10):
  # fild something from L1
  query_vec = model_predict(query)
  l1_embeds = embedding_dict['L1']
  max_sim = float('-inf')
  tarket_l1 = ''
  for k, v in l1_embeds.items():
    sim = util.pytorch_cos_sim(query_vec, v)[0][0].tolist()
    if sim > max_sim:
      max_sim = sim
      tarket_l1 = k

  # go to L2
  max_sim = float('-inf')
  target_l2 = ''
  target_l2s = []
  for k, v in embedding_dict['L2'][tarket_l1].items():
    sim = util.pytorch_cos_sim(query_vec, v)[0][0].tolist()
    if sim > max_sim:
      max_sim = sim
      target_l2 = k

    if sim > threshold:
      target_l2s.append(k)

  if threshold == 2:
    return data['L2'][tarket_l1][target_l2]
  else:
    res = ''
    for t in target_l2s:
      res += data['L2'][tarket_l1][t]

    return res

In [ ]:
from vertexai.preview.language_models import (TextEmbeddingModel,
                                              TextGenerationModel)

generation_model = TextGenerationModel.from_pretrained("text-bison@001")

In [ ]:
def ask(query):
  prompt = query_the_db(query, embedding_dict, data, 2)

  prompt_enhance = f"""Answer the question as precise as possible using the provided context. If the answer is
              not contained in the context, say "answer not available in context" \n\n
            Context: \n {prompt}?\n
            Question: \n {query} \n
            Answer:
          """
  return generation_model.predict(prompt_enhance).text

In [ ]:
ask('How can I onboard?')

'1. Create your account\n2. Add KYC and Business Address\n3. Signup Service Agreement\n4. Add your pickup locations\n5. Recharge your wallet'